<a href="https://colab.research.google.com/github/Jaat2727/Accelerate-Slow-Downloads/blob/main/Files_Downloader_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import requests
import os
import threading
import time
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript

# --- Configuration ---
DOWNLOAD_URL = "   "  # Replace with the actual download URL
DOWNLOAD_PATH = "/content/drive/MyDrive/MyDownloads"  # Replace with your desired path

# --- Mount Google Drive ---
drive.mount('/content/drive')

# --- UI Elements (enhanced styling) ---
progress_bar = widgets.IntProgress(min=0, max=100, description='Progress:', bar_style='success')
status_label = widgets.Label(value='Ready', layout=widgets.Layout(width='100%'))
speed_label = widgets.Label(value='Speed: N/A')
downloaded_label = widgets.Label(value='Downloaded: 0 MB')
total_label = widgets.Label(value='Total: 0 MB')
filename_label = widgets.Label(value='Filename: ')

def set_font_size(widget, size='16px', is_progress_bar=False):
    widget.style.description_width = 'initial'
    widget.layout.width = 'auto'
    if is_progress_bar:
        html_style = f"""
        <style>
        .widget-label {{ font-size: {size} !important; }}
        .widget-progress {{
            background-color: #f0f0f0;
            border-radius: 10px;
            overflow: hidden;
        }}
        .widget-progress .progress .progress-bar {{
            background-color: #4CAF50 !important;
            height: 30px !important;
            border-radius: 10px;
            transition: width 0.4s ease;
        }}
        .widget-progress .progress-bar span {{
            color: white;
            font-weight: bold;
            line-height: 30px;
        }}
        </style>
        """
    else:
        html_style = f'<style>.widget-label {{ font-size: {size} !important; }}</style>'
    display(widgets.HTML(html_style))

set_font_size(progress_bar, '20px', is_progress_bar=True)
set_font_size(status_label, '18px')
set_font_size(speed_label, '20px')
set_font_size(downloaded_label, '16px')
set_font_size(total_label, '16px')
set_font_size(filename_label, '16px')

ui_box = widgets.VBox([
    filename_label,
    progress_bar,
    widgets.HBox([downloaded_label, total_label]),
    speed_label,
    status_label
], layout=widgets.Layout(border='1px solid #ccc', padding='10px'))

# --- Enhanced Keep-Alive (with JavaScript and immediate termination) ---
def keep_alive(stop_event, line_spacing=15):
    js_code = """
    var intervalId = null;
    function simulateActivity() {
      document.dispatchEvent(new KeyboardEvent('keydown', {key: ' '}));
      document.dispatchEvent(new KeyboardEvent('keyup', {key: ' '}));
    }
    function startKeepAlive() {
      intervalId = setInterval(simulateActivity, 55000); // Every 55 seconds
    }
    function stopKeepAlive() {
      if (intervalId !== null) {
        clearInterval(intervalId);
        intervalId = null;
      }
    }
    startKeepAlive();
    """
    display(Javascript(js_code))

    while not stop_event.is_set():
        print("\n" * line_spacing + "Keep-alive: Simulating activity...")
        time.sleep(60)

    display(Javascript("stopKeepAlive();"))


# --- Download Function ---
def download_file(url, download_path, chunk_size=65536):
    if not os.path.exists(download_path):
        os.makedirs(download_path)

    stop_event = threading.Event()
    keep_alive_thread = threading.Thread(target=keep_alive, args=(stop_event, 15))
    keep_alive_thread.start()

    try:
        with requests.get(url, stream=True) as response:
            response.raise_for_status()

            filename = response.headers.get("Content-Disposition")
            if filename:
                filename = filename.split("filename=")[1].strip('"').strip("'")
            else:
                filename = url.split('/')[-1].split("?")[0]

            full_file_path = os.path.join(download_path, filename)
            filename_label.value = f"Filename: {filename}"

            total_size = int(response.headers.get('content-length', 0))
            total_label.value = f"Total: {total_size / (1024 * 1024):.1f} MB"
            # *** CRITICAL: Set progress_bar.max *BEFORE* the loop ***
            progress_bar.max = total_size if total_size > 0 else 100
            downloaded_size = 0
            start_time = time.time()
            previous_speed = 0

            with open(full_file_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=chunk_size):
                    if chunk:
                        file.write(chunk)
                        downloaded_size += len(chunk)
                        # *** CRITICAL: Update UI *INSIDE* the loop ***
                        previous_speed = update_ui(total_size, downloaded_size, start_time, previous_speed)

        status_label.value = f"Download complete! Saved as: {full_file_path}"
        clear_output(wait=True)
        display(ui_box)
        status_label.value = f"Download complete! Saved as: {full_file_path}"

    except requests.exceptions.RequestException as e:
        status_label.value = f"An error occurred during download: {e}"
        if 'full_file_path' in locals() and os.path.exists(full_file_path):
            os.remove(full_file_path)
    except Exception as e:
        status_label.value = f"An unexpected error occurred: {e}"
        if 'full_file_path' in locals() and os.path.exists(full_file_path):
            os.remove(full_file_path)
    finally:
        stop_event.set()
        keep_alive_thread.join()
        print("Download process finished.")

def update_ui(total_size, downloaded_size, start_time, previous_speed):
    """Updates UI, calculates/smooths speed, handles no content-length."""
    if total_size > 0:
        progress = int((downloaded_size / total_size) * 100)
        # *** CRITICAL: Update progress_bar.value *INSIDE* update_ui ***
        progress_bar.value = progress

        elapsed_time = time.time() - start_time
        if elapsed_time > 0:
            current_speed = downloaded_size / elapsed_time / (1024 * 1024)
            smoothed_speed = (current_speed + previous_speed * 4) / 5
            speed_label.value = f"Speed: {smoothed_speed:.2f} MB/s"
            previous_speed = smoothed_speed
        else:
            speed_label.value = "Speed: 0.00 MB/s"
            previous_speed = 0

        downloaded_label.value = f"Downloaded: {downloaded_size / (1024 * 1024):.1f} MB"
        return previous_speed # Return the previous speed

    else:  # Handle cases where Content-Length is not provided
        elapsed_time = time.time() - start_time
        if elapsed_time > 0:
            current_speed = downloaded_size / elapsed_time / (1024 * 1024)
            smoothed_speed = (current_speed + previous_speed * 4) / 5
            speed_label.value = f"Speed: {smoothed_speed:.2f} MB/s"
            previous_speed = smoothed_speed
        else:
            speed_label.value = "Speed: 0.00 MB/s"
            previous_speed = 0

        downloaded_label.value = f"Downloaded: {downloaded_size / (1024 * 1024):.1f} MB"
        # Cycle progress bar for unknown size
        progress_bar.value = int(downloaded_size % 100)
        return previous_speed

# --- Main Execution ---
if __name__ == "__main__":
    display(ui_box)
    download_file(DOWNLOAD_URL, DOWNLOAD_PATH, chunk_size=65536)